In [6]:

import xarray as xr
import gzip
import rasterio
import os
import urllib
from datacube.utils.cog import write_cog
from datacube.utils.geometry import assign_crs

In [11]:
def download_and_cog(year, month):
    
    #set up file strings
    filename = f"chirps-v2.0.{year}.{month}.tif"
    out_filename = f"chirps_afr_monthly/chirps-v2.0_{year}.{month}.tif"
    url = f"https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_monthly/tifs/{filename}"
    
    # download url
    if not os.path.exists(out_filename):
        response = urllib.request.urlopen(url)
    
    #unzip
    with gzip.GzipFile(fileobj=response, mode='rb') as gzip_infile:
        
        #grab tif data and metadata
        with rasterio.open(gzip_infile, 'r') as src:
            raster=src.read()
            transform=src.transform
            crs=src.crs
            height=raster.shape[1]
            width=raster.shape[2]
        
        #write to disk as geotiff
        with rasterio.open(out_filename,
                           'w',
                           driver='GTiff',
                           height=height,
                           width=width,
                           count=1,
                           dtype=rasterio.float32,
                           crs=crs,
                           transform=transform
                          ) as dst:
            dst.write(raster)
    
    #open as xarray and write as cog
    da = assign_crs(xr.open_rasterio(out_filename))
    write_cog(da,
              fname=out_filename,
              overwrite=True)


In [12]:
download_and_cog('1981', '02')

EPSG:4326


In [16]:
# xr.open_rasterio('chirps_afr_monthly/chirps-v2.0_1981.01.tif').squeeze().plot.imshow(vmin=0);